In [139]:
## Imports

# General
import os
import os.path as path
import json
from glob import glob
import cv2 as cv2
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from tqdm import tqdm

%matplotlib inline

In [138]:
!conda install -c conda-forge tqdm --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.1
  latest version: 4.10.2

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /Users/ivo/opt/anaconda3/envs/eyeBike

  added / updated specs:
    - tqdm


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    tqdm-4.61.1                |     pyhd8ed1ab_0          79 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          79 KB

The following NEW packages will be INSTALLED:

  tqdm               conda-forge/noarch::tqdm-4.61.1-pyhd8ed1ab_0



tqdm-4.61.1          | 79 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [101]:
# Set data to be processed
DROOT = './0_data'

# Sources
RND_SRC = path.join(DROOT, "2_procfootage/360-cropped.jpg")

# Destination video
VID_DST_FRMS = path.join(DROOT, "4_simsiam/video_sample/")
VID_DST = path.join(DROOT, "4_simsiam/video_sample.mp4")

In [93]:
base = plt.imread(RND_SRC)
height, width, layers = base.shape
print(base.shape)
r = 150

(1810, 6516, 3)


In [146]:
length = 100

def get_random_movement(size, main_dev, n_part, part_dev, start, bias):
    main = np.random.normal(0, main_dev, length)
    main_summed = np.array([sum(main[0:i]) + start + bias for i in range(size)])
    parts = [np.random.normal(main_summed, part_dev)
             for x in range(n_part)]
    
    return parts
    
combine_lists_of_lists = lambda x, y: [list(zip(x[i], y[i])) for i in range(len(x))]

part_1_x = get_random_movement(length, 400, 3, 100, -1600, width/2)
part_1_y = get_random_movement(length, 20, 3, 20, -100, height/2)
part_1 = combine_lists_of_lists(part_1_x, part_1_y)

part_2_x = get_random_movement(length, 100, 4, 200, 400, width/2)
part_2_y = get_random_movement(length, 20, 4, 20, 100, height/2)
part_2 = combine_lists_of_lists(part_2_x, part_2_y)

part_3_x = get_random_movement(length, 400, 4, 100, 1000, width/2)
part_3_y = get_random_movement(length, 20, 4, 20, -200, height/2)
part_3 = combine_lists_of_lists(part_3_x, part_3_y)

part_4_x = get_random_movement(length, 100, 5, 150, -700, width/2)
part_4_y = get_random_movement(length, 20, 5, 20, 400, height/2)
part_4 = combine_lists_of_lists(part_4_x, part_4_y)

parts = pd.DataFrame(part_1 + part_2 + part_3 + part_4).T

parts.head(1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"(1820.3528286178093, 802.237253463994)","(1473.1184210809156, 817.3416894723761)","(1649.28458360366, 832.1923136465618)","(3790.9419464712255, 1015.064035242685)","(3512.8538653513424, 1016.2450670048255)","(3603.7197492424793, 1034.4736777911494)","(3781.3798023212194, 1029.946524976387)","(4292.073831305268, 685.1829210705223)","(4400.097781541503, 742.2360704275967)","(4478.891034853782, 663.9141512465071)","(4396.914159223289, 745.7748257223535)","(2322.177507977992, 1302.6069426638744)","(2537.738563640068, 1305.5315184768917)","(2481.0029984417706, 1295.3589586729934)","(2413.373194059446, 1308.3838346333132)","(2399.126133931643, 1324.8605117708562)"


In [147]:
fourcc = cv2.VideoWriter_fourcc(*'FMP4')
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video = cv2.VideoWriter(VID_DST, fourcc, 10, (1440, 720))

def get_cmap(n, name='hsv'):
    return plt.cm.get_cmap(name, n)
cmap = get_cmap(len([x[0] for x in parts.iloc[0]]))

for frame in tqdm(parts.itertuples(index=False)):
    
    fig, ax = plt.subplots(figsize=(20,10))
    ax.imshow(base)
    x = [x[0] for x in frame]
    y = [x[1] for x in frame]
    ax.scatter(x, y, c=range(len(x)), cmap=cmap, linewidths=15, alpha=0.7)
    ax.axis('off')
    fig.tight_layout(pad=0, h_pad=0)
    ax.margins(0, y=0)

    fig.canvas.draw()
    image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    image_from_plot = image_from_plot.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    
    video.write(cv2.cvtColor(image_from_plot, cv2.COLOR_BGR2RGB))
    plt.close()
    
video.release()

100it [01:10,  1.42it/s]
